In [2]:
import requests
import pandas as pd
import urllib3
import logging

# 1. Silenciar o aviso de SSL/LibreSSL para limpar o terminal
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
logging.captureWarnings(True)

def extrair_dados_fies_seguro(package_id):
    api_url = f"https://dados.gov.br/api/3/action/package_show?id={package_id}"
    
    # 2. Adicionar Headers para parecer um navegador real
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        print(# "Acessando API..."
              "Acessando API...")
        response = requests.get(api_url, headers=headers, verify=False)
        
        # Verifica se a resposta é sucesso (200)
        if response.status_code != 200:
            print(f"Erro do Servidor: Status {response.status_code}")
            return

        # Tenta decodificar o JSON
        try:
            data = response.json()
        except ValueError:
            print("O servidor não retornou um JSON válido. Ele pode estar fora do ar ou bloqueando o acesso.")
            return

        resources = data['result']['resources']
        csv_url = None
        
        for resource in resources:
            if resource['format'].upper() == 'CSV':
                csv_url = resource['url']
                print(f"Link encontrado: {resource['name']}")
                break
        
        if csv_url:
            print("Baixando dados... Isso pode demorar dependendo do tamanho do arquivo.")
            # Alguns CSVs do governo usam encoding 'latin-1' e separador ';'
            df = pd.read_csv(csv_url, sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False)
            
            nome_output = "fies_dados.csv"
            df.to_csv(nome_output, index=False, encoding='utf-8-sig')
            print(f"Sucesso! Arquivo salvo como: {nome_output}")
        else:
            print("Nenhum recurso CSV disponível no momento.")

    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

ID_FIES = "ensino-superior-fies-fundo-de-financiamento-estudantil"
extrair_dados_fies_seguro(ID_FIES)

Acessando API...
O servidor não retornou um JSON válido. Ele pode estar fora do ar ou bloqueando o acesso.
